In [3]:
import pandas as pd
import numpy as np


In [55]:
df = pd.read_excel("LIG数据分类.xlsx", header=None)
df


,0,1,2,3,4,5,6,7
0,基本运行信息,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,警惕重置,是/否,喇叭电磁阀,0 = Deenergized\n1 = Energized,电空制动失效,0 = No Air Brake Failure\n1 = Air Brake Failure,反相开关,0 =前进\n1 =零位\n2 = 后退\n3 = Undefined
2,警惕报警,是/否,喇叭按钮,0 = Off 1 = On,GPS系统时间日期,yyyy-MM-dd hh:mm:ss,撒沙模式,0 =撒沙结束 \n 1 = 手动主机撒沙 \n 2 =手动列车线撒沙 \n 3 =自动...
3,警惕切除,是/否,慢速按钮,0 = Off 1 = On,GPS UTC 时间,yyyy-MM-dd hh:mm:ss,列车管线27 慢速TL27,0 = De energized\n1 = Energized
4,自动制动手柄位置,制动区/抑制/缓解/其他/紧急,导向轴撒砂开关,0 = Off 1 = On,GPS 日期时间,yyyy-MM-dd hh:mm:ss,机车档位,0 =堕转，制动设置，或制动\n 1 = 1 档 \n 2 = 2 档\n 3 = 3 档\...
...,...,...,...,...,...,...,...,...
62,故障记录日志,NaN,LIG重配置时钟状态,xxxxx Seconds,发布的C类报文组数目,NaN,NaN,NaN
63,发布的C类报文数,NaN,定义的C 类报文数,NaN,NaN,NaN,NaN,NaN
64,D类报文的有效客户端数量,NaN,定义的C类报文组数,NaN,协议版本号,NaN,NaN,NaN
65,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
vals = []
for i, s in df.iterrows():
    for i2, val in s.iteritems():
        if val is not np.nan:
            vals.append(val)


In [26]:
vals[:10]


['基本运行信息',
 '警惕重置',
 '是/否',
 '喇叭电磁阀',
 '0 = Deenergized\n1 = Energized',
 '电空制动失效',
 '0 = No Air Brake Failure\n1 = Air Brake Failure',
 '反相开关',
 '0 =前进\n1 =零位\n2 = 后退\n3 = Undefined',
 '警惕报警']

In [27]:
for i in vals:
    if "列车管线紧急制动" in i:
        print(i)


列车管线紧急制动


In [40]:
def is_code(line: str) -> bool:
    if line.startswith("float") or line.startswith("unsigned"):
        return True
    return False


In [47]:
lines = []
with open("Lig_2_0.h", encoding='utf8', mode='r') as f:
    lines = f.readlines()
lines = [line.strip() for line in lines]
lines = list(filter(is_code, lines))


In [48]:
lines


['unsigned char AlerterReset;\t\t\t\t\t\t\t//0x1001 警惕重置 1,  unsigned, 0 = No, 1 = Yes,  N/A',
 'unsigned char AlerterAlarm;\t\t\t\t\t\t\t//0x1002  警惕报警 1,  unsigned,  0 = False,  1 = True,  N/A',
 'unsigned char AlerterCutout;\t\t\t\t\t\t//0x1003 警惕切除 1,  unsigned,  0 = False 1 = True,  N/A',
 'unsigned char AutomaticBrakeHandlePos;\t\t\t\t//0x1004 自动制动手柄位置',
 'unsigned char BailOffNCommand;\t\t\t\t\t\t//0x1005  单独缓解 1,  unsigned,  0 = No Bail,1 = Bail,  N/A',
 'unsigned char ConductorValveEmergency;\t\t\t\t//0x1007 车长阀紧急 1,  unsigned,  0 = Not Applied, 1 = Applied,  N/A',
 'float DBCall;\t\t\t\t\t\t\t\t\t\t//0x1009  电阻制动百份比 4,  float,  xxx %,  0.01 | 4 字节，浮点数',
 'unsigned char DBNotch;\t\t\t\t\t\t\t\t//0x100A 电阻制动档位1,',
 'unsigned char EABBrakeSetupMode;\t\t\t\t\t//0x100F 电空制动设置模式',
 'unsigned char EABIBS;\t\t\t\t\t\t\t\t//0x1010 电空制动独立制动施加 1,  unsigned,  0 = False, less than or equal to 80%,',
 'unsigned char EABIndBrakeTrail;\t\t\t\t\t\t//0x1011  从车独立制动施加 1,  unsigned,  0 = Trail, 

In [36]:
s = set()
for i in lines:
    s.add(i.split()[0])
s


{'_ldpToRailway()',
 'float',
 'unsigned',
 '{',
 '};',
 '}LIG_INFO_2_0;',
 '}ldpToRailway;//LDP发给铁路局的'}

In [54]:
arr = []
for line in lines:
    code, comment = line.split("//", maxsplit=1)
    code = code.strip()
    if ',' in comment:
        comment = comment.split(',')[0]
    code = code.rsplit(maxsplit=1)[-1][:-1]
    arr.append([code, comment])
arr


[['AlerterReset', '0x1001 警惕重置 1'],
 ['AlerterAlarm', '0x1002  警惕报警 1'],
 ['AlerterCutout', '0x1003 警惕切除 1'],
 ['AutomaticBrakeHandlePos', '0x1004 自动制动手柄位置'],
 ['BailOffNCommand', '0x1005  单独缓解 1'],
 ['ConductorValveEmergency', '0x1007 车长阀紧急 1'],
 ['DBCall', '0x1009  电阻制动百份比 4'],
 ['DBNotch', '0x100A 电阻制动档位1'],
 ['EABBrakeSetupMode', '0x100F 电空制动设置模式'],
 ['EABIBS', '0x1010 电空制动独立制动施加 1'],
 ['EABIndBrakeTrail', '0x1011  从车独立制动施加 1'],
 ['EABPenaltyType', '0x1012  电空制动惩罚制动类型'],
 ['EABSetup', '0x1013 电空制动设置'],
 ['LCSLeadOrTrail', '0x1014 本务/重联 电空制动模式设置1'],
 ['ECSwitch', '0x1015 引擎控制开关'],
 ['EmergencyBrakesApplied', '0x1017  紧急制动施加 1'],
 ['HeadlightFront', '0x1019  前大灯开关 1'],
 ['HeadlightRear', '0x101A 后大灯开关 1'],
 ['Horn', '0x101B  喇叭压力开关 1'],
 ['LocomotiveOperatingMode', '0x101C 机车工作模式'],
 ['LocomotiveSpeed', '0x101D 机车速度4'],
 ['MeasuredFuelLevel', '0x101F 燃油液位 4'],
 ['PCSRelayStatus', '0x1021  动力切除继电器状态1'],
 ['ReverseDirection', '0x1023  反相开关1'],
 ['SandingMode', '0x1024  撒沙模式'],
 ['TL27S

In [65]:
arr2 = []
for i, row in df.iterrows():
    if len(row.dropna()) == 1:
        name = row.dropna()[0]
        arr2.append([name])
    else:
        if len(row) % 2 == 0:
            for i in range(len(row)):
                if i % 2 == 1:
                    pass
                else:
                    if not (row[i] is np.nan):
                        arr2.append([row[i], row[i+1]])
arr2


[['基本运行信息'],
 ['警惕重置', '是/否'],
 ['喇叭电磁阀', '0 = Deenergized\n1 = Energized'],
 ['电空制动失效', '0 = No Air Brake Failure\n1 = Air Brake Failure'],
 ['反相开关', '0 =前进\n1 =零位\n2 = 后退\n3 = Undefined'],
 ['警惕报警', '是/否'],
 ['喇叭按钮', ' 0 = Off  1 = On'],
 ['GPS系统时间日期', 'yyyy-MM-dd hh:mm:ss'],
 ['撒沙模式',
  '0 =撒沙结束 \n  1 = 手动主机撒沙 \n 2 =手动列车线撒沙 \n  3 =自动撒沙 \n 4 =电空制动撒沙 \n 5 = 自测试撒沙'],
 ['警惕切除', '是/否'],
 ['慢速按钮', ' 0 = Off  1 = On'],
 ['GPS UTC 时间', 'yyyy-MM-dd hh:mm:ss'],
 ['列车管线27 慢速TL27', '0 = De energized\n1 = Energized'],
 ['自动制动手柄位置', '制动区/抑制/缓解/其他/紧急'],
 ['导向轴撒砂开关', ' 0 = Off  1 = On'],
 ['GPS 日期时间', 'yyyy-MM-dd hh:mm:ss'],
 ['机车档位',
  '0 =堕转，制动设置，或制动\n 1 = 1 档 \n 2 = 2 档\n 3 = 3 档\n 4 = 4 档\n 5 = 5 档\n  6 = 6 档\n  7 = 7 档\n  8 = 8 档\n 9 =关闭\n  10 Invalid, N/A'],
 ['单独缓解', 'No Bail/Bail'],
 ['本地控制电路断路器', ' 0 = Off  1 = On'],
 ['GPS接受机状态',
  '0 = Motorola 6 Channel\n1 = Sony\n  3 = Generic NMEA Device\n4 = u-blox\n  11 = Motorola 12 Channel'],
 ['列车管线紧急制动', '0 = Not Applied  \n1 = Applied'],
 ['车长阀

In [66]:
arr3 = []
for i, val in enumerate(arr2):
    if len(val) == 1:
        l = []
        j = i+1
        if i+1 == len(arr2):
            arr3.append([val, l])
            break
        else:
            while len(arr2[j]) != 1:
                l.append(arr2[j])
                j += 1
        arr3.append([val, l])
arr3


[[['基本运行信息'],
  [['警惕重置', '是/否'],
   ['喇叭电磁阀', '0 = Deenergized\n1 = Energized'],
   ['电空制动失效', '0 = No Air Brake Failure\n1 = Air Brake Failure'],
   ['反相开关', '0 =前进\n1 =零位\n2 = 后退\n3 = Undefined'],
   ['警惕报警', '是/否'],
   ['喇叭按钮', ' 0 = Off  1 = On'],
   ['GPS系统时间日期', 'yyyy-MM-dd hh:mm:ss'],
   ['撒沙模式',
    '0 =撒沙结束 \n  1 = 手动主机撒沙 \n 2 =手动列车线撒沙 \n  3 =自动撒沙 \n 4 =电空制动撒沙 \n 5 = 自测试撒沙'],
   ['警惕切除', '是/否'],
   ['慢速按钮', ' 0 = Off  1 = On'],
   ['GPS UTC 时间', 'yyyy-MM-dd hh:mm:ss'],
   ['列车管线27 慢速TL27', '0 = De energized\n1 = Energized'],
   ['自动制动手柄位置', '制动区/抑制/缓解/其他/紧急'],
   ['导向轴撒砂开关', ' 0 = Off  1 = On'],
   ['GPS 日期时间', 'yyyy-MM-dd hh:mm:ss'],
   ['机车档位',
    '0 =堕转，制动设置，或制动\n 1 = 1 档 \n 2 = 2 档\n 3 = 3 档\n 4 = 4 档\n 5 = 5 档\n  6 = 6 档\n  7 = 7 档\n  8 = 8 档\n 9 =关闭\n  10 Invalid, N/A'],
   ['单独缓解', 'No Bail/Bail'],
   ['本地控制电路断路器', ' 0 = Off  1 = On'],
   ['GPS接受机状态',
    '0 = Motorola 6 Channel\n1 = Sony\n  3 = Generic NMEA Device\n4 = u-blox\n  11 = Motorola 12 Channel'],
   ['列车管线紧

In [67]:
arr3[0]

[['基本运行信息'],
 [['警惕重置', '是/否'],
  ['喇叭电磁阀', '0 = Deenergized\n1 = Energized'],
  ['电空制动失效', '0 = No Air Brake Failure\n1 = Air Brake Failure'],
  ['反相开关', '0 =前进\n1 =零位\n2 = 后退\n3 = Undefined'],
  ['警惕报警', '是/否'],
  ['喇叭按钮', ' 0 = Off  1 = On'],
  ['GPS系统时间日期', 'yyyy-MM-dd hh:mm:ss'],
  ['撒沙模式',
   '0 =撒沙结束 \n  1 = 手动主机撒沙 \n 2 =手动列车线撒沙 \n  3 =自动撒沙 \n 4 =电空制动撒沙 \n 5 = 自测试撒沙'],
  ['警惕切除', '是/否'],
  ['慢速按钮', ' 0 = Off  1 = On'],
  ['GPS UTC 时间', 'yyyy-MM-dd hh:mm:ss'],
  ['列车管线27 慢速TL27', '0 = De energized\n1 = Energized'],
  ['自动制动手柄位置', '制动区/抑制/缓解/其他/紧急'],
  ['导向轴撒砂开关', ' 0 = Off  1 = On'],
  ['GPS 日期时间', 'yyyy-MM-dd hh:mm:ss'],
  ['机车档位',
   '0 =堕转，制动设置，或制动\n 1 = 1 档 \n 2 = 2 档\n 3 = 3 档\n 4 = 4 档\n 5 = 5 档\n  6 = 6 档\n  7 = 7 档\n  8 = 8 档\n 9 =关闭\n  10 Invalid, N/A'],
  ['单独缓解', 'No Bail/Bail'],
  ['本地控制电路断路器', ' 0 = Off  1 = On'],
  ['GPS接受机状态',
   '0 = Motorola 6 Channel\n1 = Sony\n  3 = Generic NMEA Device\n4 = u-blox\n  11 = Motorola 12 Channel'],
  ['列车管线紧急制动', '0 = Not Applied  

In [68]:
arr3[0][0]

['基本运行信息']

In [69]:
arr3[0][1]

[['警惕重置', '是/否'],
 ['喇叭电磁阀', '0 = Deenergized\n1 = Energized'],
 ['电空制动失效', '0 = No Air Brake Failure\n1 = Air Brake Failure'],
 ['反相开关', '0 =前进\n1 =零位\n2 = 后退\n3 = Undefined'],
 ['警惕报警', '是/否'],
 ['喇叭按钮', ' 0 = Off  1 = On'],
 ['GPS系统时间日期', 'yyyy-MM-dd hh:mm:ss'],
 ['撒沙模式',
  '0 =撒沙结束 \n  1 = 手动主机撒沙 \n 2 =手动列车线撒沙 \n  3 =自动撒沙 \n 4 =电空制动撒沙 \n 5 = 自测试撒沙'],
 ['警惕切除', '是/否'],
 ['慢速按钮', ' 0 = Off  1 = On'],
 ['GPS UTC 时间', 'yyyy-MM-dd hh:mm:ss'],
 ['列车管线27 慢速TL27', '0 = De energized\n1 = Energized'],
 ['自动制动手柄位置', '制动区/抑制/缓解/其他/紧急'],
 ['导向轴撒砂开关', ' 0 = Off  1 = On'],
 ['GPS 日期时间', 'yyyy-MM-dd hh:mm:ss'],
 ['机车档位',
  '0 =堕转，制动设置，或制动\n 1 = 1 档 \n 2 = 2 档\n 3 = 3 档\n 4 = 4 档\n 5 = 5 档\n  6 = 6 档\n  7 = 7 档\n  8 = 8 档\n 9 =关闭\n  10 Invalid, N/A'],
 ['单独缓解', 'No Bail/Bail'],
 ['本地控制电路断路器', ' 0 = Off  1 = On'],
 ['GPS接受机状态',
  '0 = Motorola 6 Channel\n1 = Sony\n  3 = Generic NMEA Device\n4 = u-blox\n  11 = Motorola 12 Channel'],
 ['列车管线紧急制动', '0 = Not Applied  \n1 = Applied'],
 ['车长阀紧急', 'Not App

In [91]:
arr4 = []
for (index, mappings) in arr3:
    # print(index[0])
    df = pd.DataFrame(mappings)
    # display(df)
    arr4.append([index[0], df])
arr4

[['基本运行信息',
                  0                                                  1
  0            警惕重置                                                是/否
  1           喇叭电磁阀                     0 = Deenergized\n1 = Energized
  2          电空制动失效    0 = No Air Brake Failure\n1 = Air Brake Failure
  3            反相开关                0 =前进\n1 =零位\n2 = 后退\n3 = Undefined
  4            警惕报警                                                是/否
  5            喇叭按钮                                    0 = Off  1 = On
  6       GPS系统时间日期                                yyyy-MM-dd hh:mm:ss
  7            撒沙模式  0 =撒沙结束 \n  1 = 手动主机撒沙 \n 2 =手动列车线撒沙 \n  3 =自动...
  8            警惕切除                                                是/否
  9            慢速按钮                                    0 = Off  1 = On
  10     GPS UTC 时间                                yyyy-MM-dd hh:mm:ss
  11  列车管线27 慢速TL27                    0 = De energized\n1 = Energized
  12       自动制动手柄位置                                    制动区/抑制/缓解/

In [92]:
arr4[0][1]

,0,1
0,警惕重置,是/否
1,喇叭电磁阀,0 = Deenergized\n1 = Energized
2,电空制动失效,0 = No Air Brake Failure\n1 = Air Brake Failure
3,反相开关,0 =前进\n1 =零位\n2 = 后退\n3 = Undefined
4,警惕报警,是/否
5,喇叭按钮,0 = Off 1 = On
6,GPS系统时间日期,yyyy-MM-dd hh:mm:ss
7,撒沙模式,0 =撒沙结束 \n 1 = 手动主机撒沙 \n 2 =手动列车线撒沙 \n 3 =自动...
8,警惕切除,是/否
9,慢速按钮,0 = Off 1 = On


In [93]:
arr4[0][1].iloc[0,0]

'警惕重置'

In [94]:
arr4[0][1].iloc[0,1]

'是/否'

In [95]:
len(arr4[0][1])

52

In [96]:
df1 = arr4[0][1]

In [97]:
df1.columns

RangeIndex(start=0, stop=2, step=1)

In [98]:
df1[2] = ['']*len(df1)

In [99]:
df1

,0,1,2
0,警惕重置,是/否,
1,喇叭电磁阀,0 = Deenergized\n1 = Energized,
2,电空制动失效,0 = No Air Brake Failure\n1 = Air Brake Failure,
3,反相开关,0 =前进\n1 =零位\n2 = 后退\n3 = Undefined,
4,警惕报警,是/否,
5,喇叭按钮,0 = Off 1 = On,
6,GPS系统时间日期,yyyy-MM-dd hh:mm:ss,
7,撒沙模式,0 =撒沙结束 \n 1 = 手动主机撒沙 \n 2 =手动列车线撒沙 \n 3 =自动...,
8,警惕切除,是/否,
9,慢速按钮,0 = Off 1 = On,


In [100]:
df1.columns

Int64Index([0, 1, 2], dtype='int64')

In [101]:
df1.iloc[0,2]

''

In [102]:
arr

[['AlerterReset', '0x1001 警惕重置 1'],
 ['AlerterAlarm', '0x1002  警惕报警 1'],
 ['AlerterCutout', '0x1003 警惕切除 1'],
 ['AutomaticBrakeHandlePos', '0x1004 自动制动手柄位置'],
 ['BailOffNCommand', '0x1005  单独缓解 1'],
 ['ConductorValveEmergency', '0x1007 车长阀紧急 1'],
 ['DBCall', '0x1009  电阻制动百份比 4'],
 ['DBNotch', '0x100A 电阻制动档位1'],
 ['EABBrakeSetupMode', '0x100F 电空制动设置模式'],
 ['EABIBS', '0x1010 电空制动独立制动施加 1'],
 ['EABIndBrakeTrail', '0x1011  从车独立制动施加 1'],
 ['EABPenaltyType', '0x1012  电空制动惩罚制动类型'],
 ['EABSetup', '0x1013 电空制动设置'],
 ['LCSLeadOrTrail', '0x1014 本务/重联 电空制动模式设置1'],
 ['ECSwitch', '0x1015 引擎控制开关'],
 ['EmergencyBrakesApplied', '0x1017  紧急制动施加 1'],
 ['HeadlightFront', '0x1019  前大灯开关 1'],
 ['HeadlightRear', '0x101A 后大灯开关 1'],
 ['Horn', '0x101B  喇叭压力开关 1'],
 ['LocomotiveOperatingMode', '0x101C 机车工作模式'],
 ['LocomotiveSpeed', '0x101D 机车速度4'],
 ['MeasuredFuelLevel', '0x101F 燃油液位 4'],
 ['PCSRelayStatus', '0x1021  动力切除继电器状态1'],
 ['ReverseDirection', '0x1023  反相开关1'],
 ['SandingMode', '0x1024  撒沙模式'],
 ['TL27S

In [104]:
df1.iloc[:,0]

0              警惕重置
1             喇叭电磁阀
2            电空制动失效
3              反相开关
4              警惕报警
5              喇叭按钮
6         GPS系统时间日期
7              撒沙模式
8              警惕切除
9              慢速按钮
10       GPS UTC 时间
11    列车管线27 慢速TL27
12         自动制动手柄位置
13          导向轴撒砂开关
14         GPS 日期时间
15             机车档位
16             单独缓解
17        本地控制电路断路器
18         GPS接受机状态
19         列车管线紧急制动
20            车长阀紧急
21           本地重联关闭
22           GPS 经度
23             报警静音
24            本务/重联
25             锁轴切除
26           GPS 纬度
27           警惕声音报警
28            前大灯开关
29             手动撒沙
30           机车运行里程
31        辅助发电机现场切除
32            后大灯开关
33             环境温度
34       一致性管理器主控档位
35           电脑关闭开关
36           喇叭压力开关
37             降功指示
38      一致性管理器继电器状态
39           引擎应用状态
40           机车工作模式
41               车号
42         一致性管理器状态
43           电阻制动切除
44             机车速度
45            自负载开关
46       一致性管理器从控档位
47             燃油液位
48            自负载请求
49            司机按紧急


In [105]:
arr[0]

['AlerterReset', '0x1001 警惕重置 1']

In [106]:
for idx, i in enumerate(df1.iloc[:, 0].tolist()):
    found = False
    for (name, j) in arr:
        if i.replace(' ', '') in j.replace(' ', ''):
            found = True
            df1.iloc[idx, 2] = name
            break
    if not found:
        print(f"error finding {i}")


error finding 慢速按钮
error finding 列车管线27 慢速TL27
error finding GPS 日期时间
error finding 辅助发电机现场切除


In [107]:
df1

,0,1,2
0,警惕重置,是/否,AlerterReset
1,喇叭电磁阀,0 = Deenergized\n1 = Energized,HornMagValve
2,电空制动失效,0 = No Air Brake Failure\n1 = Air Brake Failure,AirBrakeFailure
3,反相开关,0 =前进\n1 =零位\n2 = 后退\n3 = Undefined,ReverseDirection
4,警惕报警,是/否,AlerterAlarm
5,喇叭按钮,0 = Off 1 = On,HornPushbutton
6,GPS系统时间日期,yyyy-MM-dd hh:mm:ss,GPSSystemTimeDate
7,撒沙模式,0 =撒沙结束 \n 1 = 手动主机撒沙 \n 2 =手动列车线撒沙 \n 3 =自动...,SandingMode
8,警惕切除,是/否,AlerterCutout
9,慢速按钮,0 = Off 1 = On,


In [103]:
len(arr)

202

In [115]:
found_fields = []
unfound_section = []
for section, mapping in arr4:
    mapping[2] = ['']*len(mapping)
    for idx, i in enumerate(mapping.iloc[:, 0].tolist()):
        found = False
        for (name, j) in arr:
            if i.replace(' ', '') in j.replace(' ', ''):
                found = True
                found_fields.append(name)
                mapping.iloc[idx, 2] = name
                break
        if not found:
            print(f"error finding {section}.{i}")
            unfound_section.append(i)

error finding 基本运行信息.慢速按钮
error finding 基本运行信息.列车管线27 慢速TL27
error finding 基本运行信息.GPS 日期时间
error finding 基本运行信息.辅助发电机现场切除
error finding 柴油机监控.引擎控制开关慢速
error finding 柴油机监控.引擎控制开关 运行
error finding 柴油机监控.引擎控制开关启动
error finding 柴油机监控.右侧増压器转速
error finding 辅助.列车管线12 档位B状态
error finding 辅助.TAC电流
error finding 辅助.列车管线15 档位A状态
error finding 制动.电阻制动百份比%
error finding 制动.电控制动设置模式
error finding 版本与累计.协议版本号


In [109]:
for section,mapping in arr4:
    print(section)
    display(mapping)

基本运行信息


,0,1,2
0,警惕重置,是/否,AlerterReset
1,喇叭电磁阀,0 = Deenergized\n1 = Energized,HornMagValve
2,电空制动失效,0 = No Air Brake Failure\n1 = Air Brake Failure,AirBrakeFailure
3,反相开关,0 =前进\n1 =零位\n2 = 后退\n3 = Undefined,ReverseDirection
4,警惕报警,是/否,AlerterAlarm
5,喇叭按钮,0 = Off 1 = On,HornPushbutton
6,GPS系统时间日期,yyyy-MM-dd hh:mm:ss,GPSSystemTimeDate
7,撒沙模式,0 =撒沙结束 \n 1 = 手动主机撒沙 \n 2 =手动列车线撒沙 \n 3 =自动...,SandingMode
8,警惕切除,是/否,AlerterCutout
9,慢速按钮,0 = Off 1 = On,


柴油机监控


,0,1,2
0,引擎控制开关,0 =移车 1 =发动 2 =隔离 3 =运行 4 =电阻制动,ECSwitch
1,滑油入口温度,"xxxx.x Degree , 0.1 Degree",OilInletTemp
2,TAC励磁电流,"xxx.x AMPS, 0.1 AMP",TACFieldCurrent
3,MR1 压力,"xxx.x PSI, 0.1 PSI",MainReservoirPressure1
4,引擎控制开关慢速,0 = Off 1 = On,
5,柴油机出口水温,"xxx.x Degree F, 0.1 Degree",WaterOutletTemp
6,第1轴牵引力,"xxxxx LB, 1 LB",TM1TracEffFdbk
7,LPC 发出惩罚制动指令,0 = Apply\n1 = Released,PenaltyBrakeCommand
8,引擎控制开关 运行,0 = Off 1 = On,
9,进气总管温度,"xxx.x Degree F, 0.1 Degree",MainAirTemp


牵引系统


...

,0,1,2
0,电阻制动百份比%,"xxx %, 0.01",
1,电空制动设置,"0 =客车 1 =货车 2 =切除 3 = Other, N/A",EABBrakeSetupMode
2,1轴速度,"xx.x MPH, 0.1 MPH",A1SpeedRaw
3,可用功率,"xxxx HP, 1 HP",AvailableHorsepower
4,电阻制动档位,"0 = DB Off, 1 = DB 1, 2 = DB 2, \n3 = DB 3, 4...",DBNotch
5,紧急制动施加,0 = No\n 1 = Yes,EmergencyBrakesApplied
6,2轴速度,"xx.x MPH, 0.1 MPH",A2SpeedRaw
7,电控制动设置模式,0 =补机 切除4 = 主机切除 \n 5 = Lead Cut In Vacuum\n...,
8,电阻制动功率控制,"xx.x Volts, 0.1 Volt",DBBrakePowerControl
9,3轴速度,"xx.x MPH, 0.1 MPH",A3SpeedRaw


版本与累计


,0,1,2
0,故障记录日志,NaN,IncidentLog
1,LIG重配置时钟状态,xxxxx Seconds,LIGReconfigurationTimerStatus
2,发布的C类报文组数目,NaN,NumberofPublishedClassCGroups
3,发布的C类报文数,NaN,NumberOfClassC
4,定义的C 类报文数,NaN,NumberofDefinedClassCMessages
5,D类报文的有效客户端数量,NaN,NumberOfClassDClientsActive
6,定义的C类报文组数,NaN,NumberofDefinedClassCGroups
7,协议版本号,NaN,


司机操作指令


,2


In [118]:
unused_fields = list(filter(lambda x: x[0] not in found_fields, arr))
unused_fields

[['DBCall', '0x1009  电阻制动百份比 4'],
 ['EABSetup', '0x1013 电空制动设置'],
 ['TL27SlowSpeed', '0x1025 列车管线 27 慢速 1'],
 ['AuxiliaryAlternatorFieldCutout', '0x102B 辅助发电机励磁切除 1'],
 ['ECSwitchDBOnly', '0x1032  引擎控制开关（仅电阻制动）1'],
 ['ECSwitchJog', '0x1033 引擎控制开关（移车）1'],
 ['ECSwitchRun', '0x1034  引擎控制开关（运行）1'],
 ['ECSwitchStart', '0x1035 引擎控制开关（发动）1'],
 ['GPSDateYear', '0x104B  GPS 日期 年 2'],
 ['GPSDateMonth', '0x104C GPS 日期 月 1'],
 ['GPSTimeDay', '0x104D GPS 日期 日 1'],
 ['GPSTimeHours', '0x104E GPS 时间 小时 1'],
 ['GPSTimeMinutes', '0x104F GPS 时间 分钟 1'],
 ['GPSTimeSeconds', '0x1050 GPS 时间 秒 1'],
 ['JogPushbutton', '移车按钮\t\t\t//0x1056  移车按钮 1'],
 ['TL12InputThrottleBV', '0x300B 车管线 12 档位 B 状态 1'],
 ['TL15InputThrottleAV', '0x300D 车管线 15 档位 A 状态 1'],
 ['TurboSpeedR', '2.0 更改\t\t\t\t\t\t//0x402C R侧増压器转速 4'],
 ['TAPCurrentAMPSAvg', '0x7068 TAP 电流  4']]

In [116]:
unfound_section

['慢速按钮',
 '列车管线27 慢速TL27',
 'GPS 日期时间',
 '辅助发电机现场切除',
 '引擎控制开关慢速',
 '引擎控制开关 运行',
 '引擎控制开关启动',
 '右侧増压器转速',
 '列车管线12 档位B状态',
 'TAC电流',
 '列车管线15 档位A状态',
 '电阻制动百份比%',
 '电控制动设置模式',
 '协议版本号']

In [119]:
len(unused_fields)

19

In [120]:
len(unfound_section)

14

In [131]:
lets_match = pd.DataFrame(unused_fields)
lets_match[2] = ['']*len(lets_match)
for i,v in enumerate(unfound_section):
    lets_match.iloc[i,2] = v

In [132]:

lets_match

,0,1,2
0,DBCall,0x1009 电阻制动百份比 4,慢速按钮
1,EABSetup,0x1013 电空制动设置,列车管线27 慢速TL27
2,TL27SlowSpeed,0x1025 列车管线 27 慢速 1,GPS 日期时间
3,AuxiliaryAlternatorFieldCutout,0x102B 辅助发电机励磁切除 1,辅助发电机现场切除
4,ECSwitchDBOnly,0x1032 引擎控制开关（仅电阻制动）1,引擎控制开关慢速
5,ECSwitchJog,0x1033 引擎控制开关（移车）1,引擎控制开关 运行
6,ECSwitchRun,0x1034 引擎控制开关（运行）1,引擎控制开关启动
7,ECSwitchStart,0x1035 引擎控制开关（发动）1,右侧増压器转速
8,GPSDateYear,0x104B GPS 日期 年 2,列车管线12 档位B状态
9,GPSDateMonth,0x104C GPS 日期 月 1,TAC电流


In [ ]:
df_all = []
for section,mapping in arr4:
    df_all.append(mapping)

In [137]:
errata = {
    "TAP 电流": "TAC电流",
    "引擎控制开关（运行）": "引擎控制开关 运行",
    "引擎控制开关（发动）": "引擎控制开关启动",
    "列车管线 27 慢速": "列车管线27 慢速TL27",
    "电阻制动百份比": " 电阻制动百份比%",
    "车管线 12 档位 B 状态": "列车管线12 档位B状态",
    "车管线 15 档位 A 状态":"列车管线15 档位A状态"
}

In [138]:
errata

{'TAP 电流': 'TAC电流',
 '引擎控制开关（运行）': '引擎控制开关 运行',
 '引擎控制开关（发动）': '引擎控制开关启动',
 '列车管线 27 慢速': '列车管线27 慢速TL27',
 '电阻制动百份比': ' 电阻制动百份比%',
 '车管线 12 档位 B 状态': '列车管线12 档位B状态',
 '车管线 15 档位 A 状态': '列车管线15 档位A状态'}